# Problema do Caixeiro-Viajante

O problema do Caixeiro-Viajante trata a seguinte questão: 

_Dada uma lista de cidades e a distância entre elas, qual é a rota mais curta que passa por todas as cidades e retorna à cidade de origem?_

Apesar da curta descrição, este é um dos mais famosos (e complexos) problemas na Ciência da Computação. Encontrar _a rota mais curta_ entre $n$ cidades, com algoritmos não otimizados, gasta um tempo $O(n!)$: a busca pelo caminho mais curto entre 20 cidades exige muito mais que o dobro de recursos gastos para encontrar a melhor rota entre 10 cidades.

Usar busca exaustiva entre todos os caminhos garante que o caminho mais curto será encontrado, mas só é computacionalmente viável para pequenos conjuntos de cidades. Para problemas maiores, técnicas de otimização são necessárias para realizar uma busca inteligente no espaço de estados e encontrar soluções quase-ótimas.

Abaixo, o problema será explicado matemáticamente e uma solução utilizando busca exaustiva será descrita.

### Descrição do problema usando Teoria dos Grafos

O problema do Caixeiro-Viajante pode ser modelado por grafos não direcionados com pesos, de modo que as cidades correspondam às vertices, rotas sejam as arestas, e a distância da rota seja o peso da aresta. A resposta ótima começa e termina no mesmo vértice, após todos os vértices terem sidos visitados pelo menos (ou somente) uma vez. Um modelo baseado em grafos é apresentado na Figura 01.


![Exemplo usando Grafos](imgs/img1.png "Exemplo usando Grafos")
<center style="font-size: 0.8em">Figura 01 - Exemplo de Mapa usando Grafos. Fonte: PIWONSKA, 2011</center>

### Solução utilizando Algoritmos de Busca Não-Informada (i.e. Busca Exaustiva)

#### a) Descrição do Problema
- Estados: A descrição do estado corresponde à cidade onde o Caixeiro-Viajante se encontra em um dado momento.
- Estado Inicial: Qualquer uma das cidades.
- Ações Possíveis: Ir à uma das cidades que possuem ligação com a cidade atual.
- Modelo de Transição: A lista das cidades visitadas.
- Teste de Objetivo: Todas as cidades foram visitadas e a cidade atual é a mesma de onde o Caixeiro-Viajante partiu.

####  b) Mapa
Será utilizado o mapa apresentado na Figura 01. Para representação computacional, será utilizada uma matriz $10x10$ onde cada elemento $a_{ij}$ indica a distância entre os vértices $i$ e $j$. Para indicar caminhos inexistentes será utilizado o valor $-1$, e para indicar a cidade atual será utilizado o valor $0$.

In [ ]:
dist_matrix = [
    [ 0,  8, 13, -1, -1, 14, -1,  8, -1, -1], # Cidade 1
    [ 8,  0,  9, 12, -1, -1, -1, -1, -1, 11], # Cidade 2
    [13,  9,  0, -1, 13, 15, -1, -1, -1, -1], # Cidade 3
    [-1, 12, -1,  0, 19, -1, -1, -1, -1, -1], # Cidade 4
    [-1, -1, 13, 19,  0, 15, -1, -1, -1, -1], # Cidade 5
    [14, -1, 15, -1, 15,  0, 22, 18, -1, -1], # Cidade 6
    [-1, -1, -1, -1, -1, 22,  0, -1, 21, -1], # Cidade 7
    [ 8, -1, -1, -1, -1, 18, -1,  0, 10,  8], # Cidade 8
    [-1, -1, -1, -1, -1, -1, 21, 10,  0, 12], # Cidade 9
    [-1, 11, -1, -1, -1, -1, -1,  8, 12,  0]] # Cidade 10